# Plotting CTD Data

In [ ]:
# import sys
# !conda install --yes --prefix {sys.prefix} pandas
# !conda install --yes --prefix {sys.prefix} matplotlib
# !conda install --channel conda-forge --yes --prefix {sys.prefix} gsw


In [ ]:
import pandas as pd
import numpy as np
import gsw
import matplotlib.pyplot as plt

In [ ]:
# Load in a subset so we can calculate salinity,density,depth ourselves.
data = pd.read_csv('./2021_02_08_ru34_Deployment_Orsted.cnv',
                   skiprows=np.arange(0,123), # Lets us skip the big header at the top of the file
                   names=['Temperature [°C]', # gives names to each of the columns 
                          'Conductivity [S/m]',
                          'Pressure [db]'],
                   index_col=False,           # Forces pandas to not take the first column as an index
                   delimiter=' ',             # The data is separted into columns by spaces so we have to tell pandas that
                   skipinitialspace=True,     # This tells pandas that any number of spaces means its a new value
                  usecols=[0,1,2])            # Only pull out the 1st 2nd 3rd column of data

print(data.keys())
print(data)

_Whoops!_ We should not have a negative conductivity! When we use the CTD for glider missions, we typically start the sampling on the instrument before it is in the water so it might have sampled the air for a few seconds, giving us a negative conductivity. That bad data will screw us up later so let's get rid of it!

In [ ]:
dropind = np.where(data['Conductivity [S/m]'] < 0)[0] # Find all the rows that have a negative Conductivity

data.drop(index=dropind, inplace=True) # drop the rows indicated by drop ind
data.reset_index(inplace=True, drop=True) # reset the index so the first value has index=0 and drop the old index.
print(data)

Great! Now the data is all cleaned up!

# Use the `gsw` toolbox to solve for depth, practical salinity, and density


We are going to use the `gsw` toolbox, which is preloaded with a bunch of critical equations and conversions, to calculate our salinity, density, and depth of our data from temperature, pressure, and conductivity. It is very important that the data we input into the gsw functions has the correct units that `gsw` expects.
  
Let's start with a calculation of praticial salinity. `gsw` has a function `gsw.conversions.SP_from_C` that I found by looking at the documentation for the package here https://teos-10.github.io/GSW-Python/conversions.html

In [ ]:
print(data.keys())
?gsw.conversions.SP_from_C

Under `Parameters` in the above output, we can see the expected units for each unput to the function and compare them to our units in our data. It looks like our conducitivty is in different units so I will make sure to convert that before passing it to the function. We need to convert S-->mS and m-->cm so we end up just multiplying by 10.

In [ ]:
PS = gsw.conversions.SP_from_C(data['Conductivity [S/m]']*10, 
                               data['Temperature [°C]'],
                               data['Pressure [db]'])
print(PS.head(5))


A practical salinity in the 30s is the average for the surface ocean across the planet so that looks good to me! Let's move on to finding the depth!

In [ ]:
?gsw.conversions.z_from_p

### _Side note_:  
When you see an input in documentation that has an equal sign, that means that that is the default value and we don't NEED to put an input in for those parameters. This is the case for `geo_strf_dyn_height` and `sea_surface_geopotential` in the function we are going to use.
```
Signature:
gsw.conversions.z_from_p(
    p,
    lat,
    geo_strf_dyn_height=0,
    sea_surface_geopotential=0,
```

In [ ]:
# Calculate depth from pressure

depth = gsw.conversions.z_from_p(data['Pressure [db]'], lat= 39) # I got the latitude from the header of the data file
print(depth)

In [ ]:
mean = np.mean(depth)
mean

_Woot!_ Now we have practical salinity and depth! All we have left is density and that should be one easy step!

In [ ]:
?gsw.density.sigma0

_Boy was I wrong...._  
It looks like we need absolute salinity and conservative temperature. Worry not! gsw has the tools for this.

In [ ]:
?gsw.conversions.SA_from_SP

In [ ]:
?gsw.conversions.CT_from_t

Ok so we need to find the absolute salinity first and then use that to get the conservative temperature which we then use to get the potential density. 

In [ ]:
SA = gsw.conversions.SA_from_SP(PS,data['Pressure [db]'].values, lon=-74,lat=39)

CT = gsw.conversions.CT_from_t(SA, data['Temperature [°C]'].values, data['Pressure [db]'].values)
print(CT)

In [ ]:
rho = gsw.density.sigma0(SA,CT)
print(rho)

### We made it!

Now we have our temperature, salinity, density, and depth! 
One last step, I want to add all that data into one dataframe so that it is easier to keep track of.  

In [ ]:
data['Salinity [PSU]'] = PS
data['Depth [m]']      = depth
data['Density [kg/m3]']= rho

Next we are going to plot this data so we can learn more about the water column on the day this sample was taken.  


# Plotting profiles

I like to plot in situ temperature, practical salinity, and potential density so we are going to plot those along side each other.

In [ ]:
# First let's create our plotting area
fig = plt.figure(figsize=(10.0, 10.0))

# and add panels to plot in
axes1 = fig.add_subplot(1, 3, 1)
axes2 = fig.add_subplot(1, 3, 2)
axes3 = fig.add_subplot(1, 3, 3)

`plt.figure` sets up a space for us to plot while `fig.add_suplot` adds panels to the figure `fig`.  
In `fig.add_subplot` the three numbers tell matplotlib (the number of rows, the number of columns, and the position of the axes we are making in each line of code.  

Blank axes are great and all but why dont we add some data?

In [ ]:
# First let's create our plotting area
fig = plt.figure(figsize=(10.0, 10.0))

# and add panels to plot in
axes1 = fig.add_subplot(1, 3, 1)
axes2 = fig.add_subplot(1, 3, 2)
axes3 = fig.add_subplot(1, 3, 3)

axes1.plot(data['Temperature [°C]'],data['Depth [m]'])

axes2.plot(data['Salinity [PSU]'],data['Depth [m]'])

axes3.plot(data['Density [kg/m3]'],data['Depth [m]'])

Sweet now we have lines! It looks like this data was collected as the CTD was dunked and as it was pulled back up so we cover every depth twice. I think I may have forgotten the Cardinal Rule of making a graph. There is no title or axes labels or units!

In [ ]:
# First let's create our plotting area
fig = plt.figure(figsize=(14.0, 8.0))

# and add panels to plot in
axes1 = fig.add_subplot(1, 3, 1)
axes2 = fig.add_subplot(1, 3, 2)
axes3 = fig.add_subplot(1, 3, 3)

axes1.plot(data['Temperature [°C]'],data['Depth [m]'])
axes2.plot(data['Salinity [PSU]'],data['Depth [m]'])
axes3.plot(data['Density [kg/m3]'],data['Depth [m]'])

axes1.set_ylabel('Depth [m]')
axes1.set_xlabel('Temperature [°C]')
axes1.set_title('Temperature [°C]')

axes2.set_ylabel('Depth [m]')
axes2.set_xlabel('Salinity [PSU]')
axes2.set_title('Salinity [PSU]')

axes3.set_ylabel('Depth [m]')
axes3.set_xlabel('Density [kg/m^3]')
axes3.set_title('Density [kg/m^3]')

plt.suptitle('CTD cast from 2/8/2021 off NJ')

Something still looks _wonky_ with the upper couple meters of data. Why don't we zoom in on the right side of each plot by setting our x axis limits using `axes1.set_xlim` and repeating for each axes

In [ ]:
# First let's create our plotting area
fig = plt.figure(figsize=(14.0, 8.0))

# and add panels to plot in
axes1 = fig.add_subplot(1, 3, 1)
axes2 = fig.add_subplot(1, 3, 2)
axes3 = fig.add_subplot(1, 3, 3)

axes1.plot(data['Temperature [°C]'],data['Depth [m]'])
axes2.plot(data['Salinity [PSU]'],data['Depth [m]'])
axes3.plot(data['Density [kg/m3]'],data['Depth [m]'])

axes1.set_ylabel('Depth [m]')
axes1.set_xlabel('Temperature [°C]')
axes1.set_title('Temperature [°C]')
axes1.set_xlim([5,5.7])

axes2.set_ylabel('Depth [m]')
axes2.set_xlabel('Salinity [PSU]')
axes2.set_title('Salinity [PSU]')
axes2.set_xlim([32,33.5])

axes3.set_ylabel('Depth [m]')
axes3.set_xlabel('Density [kg/m^3]')
axes3.set_title('Density [kg/m^3]')
axes3.set_xlim([25.4,26])

plt.suptitle('CTD cast from 2/8/2021 off NJ')

Now we can see some of the variability in these properties over depth! There is a slightly warmer surface layer, salinity increases with depth, and density increases with depth so the water column seems stable!  

# Plotting T-S diagram
Making a scatter plot relating Salinity and Temperature can inform us about different water masses encountered in the data. We can plot three variables on a scatter plot by assigning one variable to the x-axis, one to the y-axis, and one to the color of the scattered datapoints.

In [ ]:
fig = plt.figure(figsize=(14.0, 8.0))

plt.scatter(data['Salinity [PSU]'], data['Temperature [°C]'], s=40, c=data['Density [kg/m3]'])
plt.colorbar()

plt.title('Temperature and Salinity Plot, RU34')
plt.xlabel('Salinity [PSU]')
plt.ylabel('Temperature [°C]')

Looks like we've got some fresh, cold water and some warmer, saliter water. Let's zoom in on the warmer saltier water to see if we can decipher any more detail in that blob. We're going to use `plt.xlim` and `plt.ylim` to bound our axes, and then `vmin` and `vmax` to bound our colorbar.

In [ ]:
fig = plt.figure(figsize=(14.0, 8.0))
plt.scatter(data['Salinity [PSU]'], data['Temperature [°C]'], s=40, c=data['Density [kg/m3]'], vmin = 24, vmax = 26)
plt.colorbar()
plt.xlim([30, 34])
plt.ylim([4,5.8])

plt.title('Temperature and Salinity Plot, RU34')
plt.xlabel('Salinity [PSU]')
plt.ylabel('Temperature [°C]')

Next, we will add contour lines of constant density. <br>
To do this, we define our own function for adding contour lines using a `def` command. This function will run all of these lines of code when we input the correct parameters into `plot_TS_Contour`. This function will then `return` the variable cs, which contains our desired contour lines.

In [ ]:
def plot_TS_contours(T,S):
    mint=np.nanmin(T)
    maxt=np.nanmax(T)
    mins=np.nanmin(S)
    maxs=np.nanmax(S)
    tempL=np.linspace(mint-1,maxt+1,399)
    salL=np.linspace(mins-1,maxs+1,399)
    Tg, Sg = np.meshgrid(tempL,salL)
    sigma_theta = gsw.sigma0(Sg, Tg)+1000 # ignore effects of pressure on density
    cnt = np.linspace(sigma_theta.min(), sigma_theta.max(),399)
    cs = ax.contour(Sg, Tg, sigma_theta, colors='grey', zorder=1 ,levels=np.arange(sigma_theta.min(), sigma_theta.max()+1,1))
    cl= ax.clabel(cs,fontsize=10,inline=True,fmt='%.1f')
    return cs

Now that are function is defined, we can use it!

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
sc = plt.scatter(data['Salinity [PSU]'], data['Temperature [°C]'], s=40, c=data['Density [kg/m3]'], vmin = 24, vmax = 26)
cb = plt.colorbar()
ax.set_xlim([30,34])
ax.set_ylim([4,5.8])

plt.title('Temperature and Salinity Plot, CTD cast from 8/12/2021 off N')
plt.xlabel('Salinity [PSU]')
plt.ylabel('Temperature [°C]')
cont = plot_TS_contours( data['Temperature [°C]'],data['Salinity [PSU]'])

Nice job! Now you have plotted oceanographic data!  

Below, I want you to repeat these quality control, calculation, and plotting steps for another data file. I will load it in for you but you have to do the rest! Use these notes to help you out along the way!

# Now you do it!
Use the code below to load in a different CTD cast (this one is from August). <br>
(1) Remove the header and the data from before the instrument was in the water. <br>
(2) Calculate density and depth, then plot profiles for temperature, salinity, and density as we did above. <br>
(3) Plot a T-S diagram for this CTD cast. <br>
(4) How does this cast differ from the one above? How is it the similar? What are some potential reasons from these similarities / differences?

In [ ]:
data2 = pd.read_csv('./2021_08_12_mara02_recovery.cnv',
                   skiprows=np.arange(0,124), # Lets us skip the big header at the top of the file
                   names=['Temperature [°C]', # gives names to each of the columns 
                          'Conductivity [S/m]',
                          'Pressure [db]'],
                   index_col=False,           # Forces pandas to not take the first column as an index
                   delimiter=' ',             # The data is separted into columns by spaces so we have to tell pandas that
                   skipinitialspace=True,     # This tells pandas that any number of spaces means its a new value
                  usecols=[0,1,2])            # Only pull out the 1st 2nd 3rd column of data

print(data2.keys())
print(data2)